In [1]:
import pandas as pd
import polars as pl
import numpy as np
import lightgbm as lgb
import optuna
import os

In [2]:
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_03_fe_v03.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

semillas = [122219, 109279, 400391, 401537, 999961]

In [3]:
data = pl.scan_parquet(dataset_path + dataset_file).filter(pl.col("foto_mes") == 202109).collect()

In [4]:
# convirto a pandas dataframe
data = data.to_pandas()

In [5]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [6]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [7]:
X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)

### Predicciones para Kaggle

In [16]:
version = 'v003' # UPADTE

modelos = [f'lgb_competencia3_{version}_s{semilla}_final.txt' for semilla in semillas]

# probamos varios puntos de corte
puntos_corte = np.linspace(10000, 14000, 10).astype(int)
# puntos_corte = [12500,13000]

In [17]:
puntos_corte

array([10000, 10444, 10888, 11333, 11777, 12222, 12666, 13111, 13555,
       14000])

In [18]:
len(modelos)

5

In [19]:
os.makedirs(base_path + f'exp/competencia_3/{version}/', exist_ok= True) 

counter = 0

for modelo in modelos:
    
    # cargamos el modelo
    model = lgb.Booster(model_file=f'{modelos_path}{version}/{modelo}')
    
    # predecimos para septiembre
    X_test = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1).copy()
    y_pred_lgm = model.predict(X_test)
    X_test['pred_lgm'] = y_pred_lgm
    
    # ordeno de mayor probabilidad de baja a menor
    idx = np.argsort(y_pred_lgm)[::-1]
    X_test.reset_index(drop=True, inplace=True)
    X_test = X_test.iloc[idx]
    
    for corte in puntos_corte:
        
        envios = np.zeros(len(X_test), dtype=int)
        envios[:corte] = 1
        X_test['Predicted'] = envios
        
        output = X_test[['numero_de_cliente', 'Predicted']]
        
        counter += 1
        file_name = f'results_{version}_{counter}.csv'
        output_path = base_path + f'exp/competencia_3/{version}/' + file_name
        
        output.to_csv(output_path, index=False)
                
        print(f'{output_path} --- modelo: {modelo}, corte: {corte}')

C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v003/results_v003_1.csv --- modelo: lgb_competencia3_v003_s122219_final.txt, corte: 10000
C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v003/results_v003_2.csv --- modelo: lgb_competencia3_v003_s122219_final.txt, corte: 10444
C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v003/results_v003_3.csv --- modelo: lgb_competencia3_v003_s122219_final.txt, corte: 10888
C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v003/results_v003_4.csv --- modelo: lgb_competencia3_v003_s122219_final.txt, corte: 11333
C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v003/results_v003_5.csv --- modelo: lgb_competencia3_v003_s122219_final.txt, corte: 11777
C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v003/results_v003_6.csv --- modelo: lgb_competencia3_v003_s122219_final.txt, corte: 12222
C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v003/results_v003_7.csv --- modelo: lgb_competencia3_v003_s122219_final.txt, corte: 12666
C:/Eugenio/Maestria/DMEyF/exp/competencia_3/v003/results_v003_